<a href="https://colab.research.google.com/github/phuocnguyen90/Random-projects/blob/main/Receipt_OCR_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hướng dẫn nhanh sử dụng công cụ trích xuất hóa đơn

File Google Colab này giúp bạn đọc thông tin từ ảnh hóa đơn và lưu vào Google Trang tính (Google Sheet).

**Các bước thực hiện:**

1.  **Chạy ô code #1 (Install dependencies):**
    *   Tìm ô code đầu tiên có tiêu đề `Install dependencies`.
    *   Nhấn nút ▶️ (Play) bên trái ô code đó để bắt đầu cài đặt.
    *   **Chờ** cho ô code chạy xong (có thể mất vài phút, đến khi biểu tượng ngừng quay hoặc có dấu tick xanh).

2.  **Chạy ô code #2 (Authenticate & Setup):**
    *   Tìm ô code thứ hai (có tiêu đề `# Authenticate...`).
    *   Nhấn nút ▶️ bên trái ô code.
    *   **Nhập thông tin được yêu cầu** khi được hỏi:
        *   `OpenRouter API Key`: Cần có để công cụ hiểu và trích xuất thông tin hóa đơn. Bạn có thể lấy key miễn phí tại [https://openrouter.ai/keys](https://openrouter.ai/keys).
        *   `(Các API Key khác nếu có)`: Cần có Ngrok API (miễn phí tại https://ngrok.com/) để xuất đường link đến giao diện web. Có thể dùng LLamaparse API key (https://www.llamaindex.ai/llamaparse) để thực hiện OCR với độ chính xác cao hơn
        *   `Tên Google Sheet`: Gõ **tên chính xác** của bảng tính Google Sheet bạn muốn dùng để lưu kết quả. Bảng tính này phải thuộc tài khoản Google bạn sẽ dùng ở bước sau.
    *   Một **cửa sổ Google sẽ hiện ra**. Chọn tài khoản Google của bạn và nhấn **"Cho phép" (Allow)** để cấp quyền truy cập Google Sheet.
    
    *   Lưu ý bảo mật: Đối với đoạn code viết sẵn này **dữ liệu từ Google Sheet của bạn được bảo mật bởi chính Google. Tuy nhiên nội dung hóa đơn được gửi qua các dịch vụ AI khác (Gwen,Meta,Gemma,Deepseek bản free). Cần cân nhắc sử dụng**. Bạn có thể tùy chỉnh code để sử dụng ChatGPT API hoặc local LLM để đảm bảo bảo mật dữ liệu hơn.

    *   Chờ ô code chạy xong và báo thành công (`✅ .env file created successfully`).

3.  **Chạy ô code #3 (Write app.py):**
    *   Nhấn nút ▶️ bên trái ô code thứ ba để tạo file ứng dụng. Bước này thường chạy rất nhanh.

4.  **Chạy ô code #4 (Run Streamlit):**
    *   Nhấn nút ▶️ bên trái ô code cuối cùng để khởi động ứng dụng web.
    *   Chờ một lát, bạn sẽ thấy một dòng chữ như: `Your Streamlit app should be available at: https://....ngrok-free.app`

5.  **Sử dụng Ứng dụng Web:**
    *   **Nhấp vào đường link `.app`** vừa xuất hiện ở trên để mở ứng dụng trong tab mới.
    *   Trong trang web đó:
        *   **Tải lên (Upload)** file ảnh hóa đơn hoặc PDF từ máy tính của bạn ở cột bên trái.
        *   Chọn phương thức OCR (thường để mặc định là Tesseract).
        *   Nhấn nút **"1. Run OCR"**. Chờ kết quả OCR xuất hiện.
        *   Nhấn nút **"2. Extract Data with LLM"**. Chờ thông tin được trích xuất ở cột bên phải.
        *   **Xem lại và chỉnh sửa** thông tin (nếu cần) trong các ô ở cột bên phải.
        *   Cuộn xuống dưới cùng cột phải và nhấn nút **"Confirm & Save to Google Sheet"** để lưu dữ liệu vào Google Sheet của bạn.

**Lưu ý:** Nếu ứng dụng báo lỗi kết nối Google Sheet, hãy quay lại Colab, chạy lại **ô code #2** và **ô code #4**.

In [ ]:
# @title 1. Install dependencies (takes 1-2 mins)
%%capture
# Install system dependencies for Tesseract OCR and PDF processing
!sudo apt-get update
!sudo apt-get install tesseract-ocr tesseract-ocr-vie poppler-utils -y

# Install Python libraries
!pip install streamlit pyngrok pytesseract pdf2image paddleocr paddlepaddle openai gspread google-auth google-auth-oauthlib python-dotenv llama-parse Pillow -q

# Pillow is usually pre-installed, but explicit install ensures it's there.
# Use paddlepaddle (CPU) for simplicity in Colab unless you have a GPU runtime and want GPU acceleration (paddlepaddle-gpu)
# llama-parse is used here as an example API parser. Replace/add others if needed.
# openai is used for the LLM extraction part. Replace/add others (e.g., anthropic, google-generativeai) if needed.

In [ ]:
# @title 2. Authenticate Google User & Setup Environment (Explicit Credential Handling)
# Cell 2: Use Colab's pop-up, explicitly get credentials, validate access

import os
from getpass import getpass
import gspread
from google.colab import auth as colab_auth # Use Colab's auth mechanism
import google.auth # <--- Import google.auth to explicitly get credentials
from google.auth.exceptions import DefaultCredentialsError
import openai
import sys

# Define scopes required by Sheets and Drive APIs
SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]

print("--- User Input & Google Authentication ---")
print("Please enter API keys and the Google Sheet name.")
print("You will then be prompted via a pop-up to authenticate with Google.")

# --- 1. Get User Inputs ---
openrouter_api_key = getpass('Enter your OpenRouter API Key: ')
llamaparse_api_key = getpass('Enter your LlamaParse API Key (optional): ')
ngrok_auth_token = getpass('Enter your NGROK Auth Token (optional): ')
google_sheet_name = input('Enter the exact name of your Google Sheet: ')

# --- 2. Perform Google Authentication via Colab ---
print("\n--- Authenticating Google User via Colab Pop-up ---")
colab_auth_ok = False
try:
    colab_auth.authenticate_user() # Trigger the pop-up
    print("✅ Google User authenticated via Colab pop-up.")
    colab_auth_ok = True
except Exception as e:
    print(f"❌ ERROR: Colab authentication failed: {e}")

# --- 3. Explicitly Get Credentials & Validate Sheet Access ---
gsheet_access_ok = False
apis_enabled_ok = True
gspread_client = None # Initialize client variable

if colab_auth_ok and google_sheet_name:
    print(f"\n--- Validating Access to Google Sheet: '{google_sheet_name}' ---")
    try:
        # *** Explicitly get default credentials set by Colab auth ***
        print("Attempting to fetch default credentials...")
        credentials, project_id = google.auth.default(scopes=SCOPES)
        print(f"✅ Successfully fetched default credentials (Project ID: {project_id}).")

        # *** Authorize gspread with the explicit credentials ***
        print("Authorizing gspread client...")
        gc = gspread.authorize(credentials)
        print("✅ gspread client authorized.")
        gspread_client = gc # Store client for potential reuse if needed

        # *** Open the sheet ***
        print(f"Opening Google Sheet '{google_sheet_name}'...")
        spreadsheet = gc.open(google_sheet_name)
        _ = spreadsheet.sheet1.title # Access a property to confirm
        gsheet_access_ok = True
        print(f"✅ Successfully accessed Google Sheet '{google_sheet_name}'.")

    except DefaultCredentialsError as e:
        # This error means google.auth.default() failed, even after Colab auth
        print(f"❌ ERROR: Could not get default credentials after Colab auth: {e}")
        print("   Try re-running this cell. If it persists, the Colab environment might have an issue.")
        colab_auth_ok = False # Mark as failed if we can't get creds
    except gspread.exceptions.SpreadsheetNotFound:
        print(f"❌ ERROR: Google Sheet '{google_sheet_name}' not found OR not shared with the authenticated Colab user.")
    except gspread.exceptions.APIError as e:
        apis_enabled_ok = False
        print(f"❌ ERROR: Google API Error accessing sheet: {e}")
        if '403' in str(e) or 'PERMISSION_DENIED' in str(e):
             print("   (Ensure Sheets & Drive APIs are enabled in Google Cloud and the user has permissions).")
    except Exception as e:
        # Catch other potential errors during authorize or open
        print(f"❌ ERROR: Failed during gspread authorization or sheet opening: {type(e).__name__} - {e}")

elif not google_sheet_name:
     print("ℹ️ INFO: Google Sheet Name not provided. Skipping sheet access validation.")
elif not colab_auth_ok:
     print("ℹ️ INFO: Skipping sheet access validation due to failed Colab authentication.")


# --- 4. API Key Validation (OpenRouter & LlamaParse) ---
# (This section remains the same as before)
print("\n--- API Key Validation ---")
openrouter_ok = False
if openrouter_api_key:
    print("Checking OpenRouter API Key presence...")
    openrouter_ok = True
    print("✅ OpenRouter API Key provided (basic check).")
else:
    print("ℹ️ INFO: OpenRouter API Key not provided. LLM extraction will be disabled.")
    openrouter_ok = True # Allow proceeding if key is not provided

llamaparse_provided = bool(llamaparse_api_key)
if llamaparse_provided: print("ℹ️ INFO: LlamaParse API Key provided (basic check only).")
else: print("ℹ️ INFO: LlamaParse API Key not provided. LlamaParse option will be disabled.")


# --- 5. Write .env File ---
print("\n--- Summary & .env Creation ---")

# Essentials: Colab Auth must succeed AND we must be able to get default creds,
# AND ( (sheet name provided AND sheet access ok) OR sheet name not provided )
# AND APIs must be enabled (or presumed enabled if access not attempted)
essentials_ok = colab_auth_ok and \
                (gsheet_access_ok or not google_sheet_name) and \
                apis_enabled_ok

if essentials_ok:
    if google_sheet_name and not gsheet_access_ok: print("⚠️ WARNING: Google Sheet access validation failed. Streamlit app might fail.")
    elif not google_sheet_name: print("ℹ️ INFO: No Google Sheet name provided. Export disabled.")
    else: print("✅ Colab authentication and Sheet access validation successful.")

    if not openrouter_ok and openrouter_api_key: print("⚠️ WARNING: OpenRouter API Key validation skipped/failed.")

    try:
        with open(".env", "w") as f:
            if openrouter_api_key: f.write(f"OPENROUTER_API_KEY={openrouter_api_key}\n")
            if llamaparse_api_key: f.write(f"LLAMAPARSE_API_KEY={llamaparse_api_key}\n")
            if ngrok_auth_token: f.write(f"NGROK_AUTH_TOKEN={ngrok_auth_token}\n")
            if google_sheet_name: f.write(f"GOOGLE_SHEET_NAME={google_sheet_name}\n")

        print("✅ .env file created successfully.")
        print("You can now proceed to run Cell 3 (Write app.py) and Cell 4 (Run Streamlit).")
        print("NOTE: If the Streamlit app later shows Google connection errors,")
        print("      you may need to re-run this cell to refresh Colab authentication.")

    except Exception as e:
        print(f"❌ ERROR: Failed to write .env file: {e}")

else:
    print("❌ ERROR: Essential validation failed. Cannot write .env file.")
    print("Please review the errors above and rerun this cell.")
    if not colab_auth_ok: print("   - Issue: Google authentication via Colab failed OR could not fetch default credentials after auth.")
    # The error 'Sheet not found...' now implies colab_auth_ok was true, but access failed later
    if colab_auth_ok and google_sheet_name and not gsheet_access_ok:
        # Distinguish between API not enabled and sheet not found/shared
        if not apis_enabled_ok:
            print("   - Issue: Google Sheets/Drive API likely not enabled or permission issue detected.")
        else:
            print("   - Issue: Google Sheet not found or not shared correctly with the authenticated user.")
    # Check OpenRouter only if it was the reason for failure (less likely)
    # if not openrouter_ok: print("   - Issue: OpenRouter validation failed (if applicable).")

--- User Input & Google Authentication ---
Please enter API keys and the Google Sheet name.
You will then be prompted via a pop-up to authenticate with Google.
Enter your OpenRouter API Key: ··········
Enter your LlamaParse API Key (optional): ··········
Enter your NGROK Auth Token (optional): ··········
Enter the exact name of your Google Sheet: receipt

--- Authenticating Google User via Colab Pop-up ---
✅ Google User authenticated via Colab pop-up.

--- Validating Access to Google Sheet: 'receipt' ---
Attempting to fetch default credentials...
✅ Successfully fetched default credentials (Project ID: ).
Authorizing gspread client...
✅ gspread client authorized.
Opening Google Sheet 'receipt'...
✅ Successfully accessed Google Sheet 'receipt'.

--- API Key Validation ---
Checking OpenRouter API Key presence...
✅ OpenRouter API Key provided (basic check).
ℹ️ INFO: LlamaParse API Key not provided. LlamaParse option will be disabled.

--- Summary & .env Creation ---
✅ Colab authentication 

In [ ]:
# @title 3. Write app.py
%%writefile app.py

import streamlit as st
import os
import json
from PIL import Image
import pytesseract
from pdf2image import convert_from_bytes
from paddleocr import PaddleOCR
import openai
import gspread
import google.auth # <--- Import google.auth
from google.auth.exceptions import DefaultCredentialsError
from llama_parse import LlamaParse
from dotenv import load_dotenv
import io
import datetime
import pandas as pd
import numpy as np
import asyncio



# --- Page Config MUST BE THE FIRST STREAMLIT COMMAND ---
st.set_page_config(layout="wide")

# --- Configuration & Initialization ---
load_dotenv()

# --- MODIFIED: Load OpenRouter Key ---
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
# --- End Modification ---
LLAMAPARSE_API_KEY = os.getenv("LLAMAPARSE_API_KEY")
GOOGLE_SHEET_NAME = os.getenv("GOOGLE_SHEET_NAME")
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

# --- Add OpenRouter Configuration ---
OPENROUTER_BASE_URL = "https://openrouter.ai/api/v1"

FREE_ROUTER_MODELS = [
    "qwen/qwen2.5-vl-32b-instruct:free", # Most likely to support JSON mode from this list
    "meta-llama/llama-4-scout:free",
    "google/gemma-3-12b-it:free",
    "deepseek/deepseek-v3-base:free",
]

# Define the model to use on OpenRouter (must match their naming)
# Examples: "openai/gpt-4o", "google/gemini-pro-1.5", "anthropic/claude-3-haiku"
# Let's default to gpt-4o via OpenRouter
DEFAULT_OPENROUTER_MODEL = os.getenv("OPENROUTER_MODEL", "meta-llama/llama-4-scout:free")

# Define scopes needed by gspread
GSPREAD_SCOPES = ["https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]


# --- Initialize OpenRouter Client (using openai library structure) ---
openrouter_client = None
if OPENROUTER_API_KEY:
    try:

        openrouter_client = openai.OpenAI(
            base_url=OPENROUTER_BASE_URL,
            api_key=OPENROUTER_API_KEY,
            # default_headers=headers # Uncomment to add headers
        )
        print("OpenRouter client initialized.") # Log for server console
    except Exception as e:

        st.error(f"Failed to initialize OpenRouter client: {e}", icon="❌")
# --- End Client Initialization ---


# --- Helper Functions (OCR, PDF, LlamaParse remain the same) ---

def process_image_tesseract(image_bytes):
    """Performs OCR on image bytes using Tesseract for Vietnamese."""
    try:
        image = Image.open(io.BytesIO(image_bytes))
        text = pytesseract.image_to_string(image, lang='vie')
        return text
    except Exception as e:
        st.error(f"Tesseract OCR failed: {e}")
        return None

def process_pdf_tesseract(pdf_bytes):
    """Converts PDF to images and performs OCR using Tesseract for Vietnamese."""
    text = ""
    try:
        images = convert_from_bytes(pdf_bytes)
        for i, image in enumerate(images):
            # st.write(f"Processing PDF page {i+1} with Tesseract (Vietnamese)...") # Reduce noise
            text += pytesseract.image_to_string(image, lang='vie') + "\n\n"
        return text
    except Exception as e:
        st.error(f"Tesseract PDF processing failed: {e}")
        return None

def process_image_paddle(image_bytes, paddle_ocr_instance):
    """Performs OCR on image bytes using PaddleOCR (already configured for Vietnamese)."""
    if paddle_ocr_instance is None:
        st.error("PaddleOCR is not initialized.")
        return None
    try:
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        image_np = np.array(image)
        result = paddle_ocr_instance.ocr(image_np, cls=True)
        text = ""
        if result and result[0]:
            for line in result[0]:
                text += line[1][0] + "\n"
        return text
    except Exception as e:
        st.error(f"PaddleOCR failed: {e}")
        return None

def process_pdf_paddle(pdf_bytes, paddle_ocr_instance):
    """Converts PDF to images and performs OCR using PaddleOCR (already configured for Vietnamese)."""
    if paddle_ocr_instance is None:
        st.error("PaddleOCR is not initialized.")
        return None
    text = ""
    try:
        images = convert_from_bytes(pdf_bytes)
        for i, image in enumerate(images):
            # st.write(f"Processing PDF page {i+1} with PaddleOCR (Vietnamese)...") # Reduce noise
            image_np = np.array(image.convert("RGB"))
            result = paddle_ocr_instance.ocr(image_np, cls=True)
            if result and result[0]:
                for line in result[0]:
                    text += line[1][0] + "\n"
            text += "\n\n" # Add separator between pages
        return text
    except Exception as e:
        st.error(f"PaddleOCR PDF processing failed: {e}")
        return None

def process_file_llamaparse(file_bytes, filename, parser_instance):
    """Uses LlamaParse API to extract text from image or PDF."""
    if parser_instance is None:
        # Error/info message handled in sidebar based on key presence/init status
        # st.error("LlamaParse parser not initialized.")
        return None
    temp_filepath = None
    try:
        # Ensure temp file has extension if needed by parser
        _, extension = os.path.splitext(filename)
        temp_filepath = f"./temp_llamaparse{extension}"
        with open(temp_filepath, "wb") as f:
            f.write(file_bytes)

        # Handle asyncio event loop for Streamlit
        try:
            loop = asyncio.get_event_loop_policy().get_event_loop()
            # Use loop.is_running() check if needed, but run_until_complete usually handles it
        except RuntimeError:
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)

        # Await the async call correctly
        documents = loop.run_until_complete(parser_instance.aload_data(temp_filepath))

        if documents:
            # Combine text from potentially multiple documents LlamaParse might return
            full_text = "\n\n".join([doc.text for doc in documents])
            return full_text
        else:
             st.warning("LlamaParse returned no documents.", icon="⚠️")
             return None
    except Exception as e:
        st.error(f"LlamaParse API call failed: {e}", icon="❌")
        return None
    finally:
        if temp_filepath and os.path.exists(temp_filepath):
            try:
                os.remove(temp_filepath)
            except OSError as e:
                st.warning(f"Could not remove temporary file {temp_filepath}: {e}", icon="⚠️")

# --- MODIFIED: extract_data_with_llm function (Model Routing Logic) ---
def extract_data_with_llm(text):
    """Uses OpenRouter with model routing to extract structured data."""
    global openrouter_client
    if not openrouter_client:
        st.error("OpenRouter client not initialized. Check API Key.", icon="❗")
        return None
    if not text or not text.strip():
        st.warning("No text provided for extraction.", icon="⚠️")
        return None

    # The prompt remains the same
    prompt = f"""
    You are an expert assistant specialized in extracting information from Vietnamese receipts.
    The following text was extracted from a receipt, likely in Vietnamese.
    Extract the key information and format the output as a single JSON object.
    Use the exact English keys provided below. If a value is not found, use null or an empty string "".

    Keys to extract:
    - 'buyer_name': Name of the customer/buyer (Tên khách hàng).
    - 'buyer_address': Address of the customer/buyer (Địa chỉ khách hàng).
    - 'buyer_contact': Phone number or email of the customer/buyer (SĐT/Email khách hàng).
    - 'receipt_date': Date the receipt was issued (Ngày hóa đơn). Format this as YYYY-MM-DD. If the date is like DD/MM/YYYY or DD-MM-YYYY, convert it.
    - 'store_name': Name of the store/vendor (Tên cửa hàng / Đơn vị bán).
    - 'store_address': Address of the store/vendor (Địa chỉ cửa hàng).
    - 'total_amount': The final total amount paid (Tổng cộng / Tổng thanh toán). Provide only the numerical value, removing currency symbols like 'đ' or 'VND' and thousand separators like '.' or ','.
    - 'items': A list of items purchased. Each item MUST be an object with 'description' (Tên hàng / Mô tả), 'quantity' (Số lượng - SL), and 'price' (Đơn giá or Thành tiền). Extract numerical values for quantity and price.

    Important Notes:
    - The text is in Vietnamese. Pay attention to Vietnamese names, addresses, and date formats (DD/MM/YYYY).
    - For 'total_amount', 'quantity', and 'price', extract only numbers. Handle separators (like '.' for thousands in VND) correctly. For example, '50.000 đ' should become 50000.
    - Output only the JSON object, nothing else before or after it.

    Receipt Text (Vietnamese):
    ---
    {text}
    ---

    JSON Output:
    """

    extracted_data = None
    last_error = None
    last_error_model = None
    successful_model = None

    # Optionally shuffle the list each time to distribute load
    # current_model_list = random.sample(FREE_ROUTER_MODELS, len(FREE_ROUTER_MODELS))
    current_model_list = FREE_ROUTER_MODELS # Or use the fixed order

    st.info(f"Attempting extraction using free models: {', '.join(current_model_list)}")

    for model_name in current_model_list:
        st.write(f"Trying model: `{model_name}`...") # Give feedback on attempts
        content = None # Reset content for each model attempt
        try:
            response = openrouter_client.chat.completions.create(
                model=model_name, # Use the current model from the list
                messages=[
                    {"role": "system", "content": "You are an expert receipt data extraction assistant specializing in Vietnamese documents."},
                    {"role": "user", "content": prompt}
                ],
                response_format={"type": "json_object"}, # Attempt JSON mode
                temperature=0.1,
                # Add a timeout? e.g., timeout=30.0
            )
            content = response.choices[0].message.content

            # Try parsing the JSON
            try:
                extracted_data = json.loads(content)
                # Basic validation: Check if it's a dictionary
                if isinstance(extracted_data, dict):
                    st.success(f"✅ Successfully extracted data using `{model_name}`!")
                    successful_model = model_name
                    break # Exit loop on first successful extraction and parse
                else:
                    st.warning(f"⚠️ Model `{model_name}` returned valid JSON, but it wasn't a dictionary. Trying next.", icon="⚠️")
                    last_error = f"Returned non-dict JSON: {content[:100]}..."
                    last_error_model = model_name
                    extracted_data = None # Reset data as it wasn't the expected type

            except json.JSONDecodeError:
                st.warning(f"⚠️ Model `{model_name}` returned invalid JSON. Trying next.", icon="⚠️")
                # Optionally show the invalid JSON
                # with st.expander(f"Invalid JSON from {model_name}"):
                #    st.code(content, language=None)
                last_error = f"Invalid JSON: {content[:100]}..."
                last_error_model = model_name
                # Keep extracted_data as None and continue loop

        except openai.AuthenticationError as e:
             st.error("OpenRouter Authentication Error: Invalid API Key?", icon="❗")
             return None # Auth errors are fatal, no point trying other models
        except openai.RateLimitError as e:
             st.warning(f"⏳ Rate limit hit for `{model_name}`. Trying next...", icon="⏳")
             last_error = e
             last_error_model = model_name
             continue # Try the next model
        except openai.APITimeoutError as e:
             st.warning(f"⏳ Timeout connecting to `{model_name}`. Trying next...", icon="⏳")
             last_error = e
             last_error_model = model_name
             continue
        except openai.APIError as e: # Catch other OpenRouter/model-specific API errors
            st.warning(f"❌ API Error with `{model_name}` (Code: {e.status_code}): {e.message}. Trying next...", icon="❌")
            last_error = e
            last_error_model = model_name
            continue # Try the next model
        except Exception as e:
            st.warning(f"⚠️ Unexpected error with `{model_name}`: {type(e).__name__}. Trying next...", icon="⚠️")
            last_error = e
            last_error_model = model_name
            continue # Try the next model

    # After the loop completes
    if successful_model:
        return extracted_data # Return the successfully parsed data
    else:
        st.error("❌ All attempted free models failed to extract valid data.", icon="❌")
        if last_error:
            st.caption(f"Last error encountered (with `{last_error_model}`): {last_error}")
        return None # Indicate failure

# --- Google Sheet Functions (get_gspread_client, connect_to_gsheet, append_to_gsheet remain the same) ---
@st.cache_resource(ttl=600)
def get_gspread_client():
    """Attempts to get an authorized gspread client using explicit default credentials."""
    try:
        # Explicitly get default credentials set by Colab auth
        credentials, project = google.auth.default(scopes=GSPREAD_SCOPES)
        # *** Authorize gspread with the explicit credentials ***
        client = gspread.authorize(credentials) # <--- THE CORRECTION
        st.success("Successfully obtained Google API client via Colab credentials.", icon="✅")
        return client
    except DefaultCredentialsError:
        st.error("❌ Google Credentials not found by application.", icon="🚨")
        st.warning("💡 Please run the 'Authenticate Google User' cell in Colab and RESTART this Streamlit app.", icon="💡")
        return None
    except Exception as e:
        st.error(f"❌ Failed to initialize Google client: {type(e).__name__} - {e}", icon="🚨")
        return None

def connect_to_gsheet():
    """Connects to the specified Google Sheet."""
    if not GOOGLE_SHEET_NAME: st.warning("Sheet Name missing.", icon="⚠️"); return None, None
    client = get_gspread_client() # Uses the corrected function
    if client:
        try: spreadsheet = client.open(GOOGLE_SHEET_NAME); sheet = spreadsheet.sheet1; return sheet, client
        except gspread.exceptions.SpreadsheetNotFound: st.error(f"❌ Sheet '{GOOGLE_SHEET_NAME}' not found/shared.", icon="🚨"); return None, None
        except gspread.exceptions.APIError as e: st.error(f"❌ Google API Error: {e}.", icon="🚨"); return None, None
        except Exception as e: st.error(f"❌ Failed to open Sheet: {e}", icon="🚨"); return None, None
    else: return None, None # Error handled in get_gspread_client


def append_to_gsheet(sheet, data):
    """Appends extracted data as a new row in the Google Sheet."""
    # ... (This function's logic remains exactly the same) ...
    try:
        headers = [
            'Extraction Date', 'Buyer Name', 'Buyer Address', 'Buyer Contact',
            'Receipt Date', 'Store Name', 'Store Address', 'Total Amount',
            'Items JSON'
        ]
        header_row = []
        try:
            header_row = sheet.row_values(1)
        except gspread.exceptions.APIError as e:
             if 'PERMISSION_DENIED' in str(e) or '403' in str(e):
                 st.warning(f"Could not read header row (Permission Denied: {e}). Assuming headers exist or sheet is empty. Attempting to append.", icon="⚠️")
             else:
                st.warning(f"Could not read header row ({e}). Assuming sheet is empty.", icon="⚠️")
                header_row = []

        if not header_row:
             try:
                 sheet.append_row(headers, value_input_option='USER_ENTERED')
                 st.info("Added header row to Google Sheet.", icon="ℹ️")
             except gspread.exceptions.APIError as append_e:
                 st.error(f"Failed to add header row: {append_e}. Check sheet write permissions.", icon="❌")
                 return False
        elif header_row != headers:
             st.warning("Sheet headers don't match expected headers. Appending based on defined order. Please check your Google Sheet columns.", icon="⚠️")

        row_to_append = [
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            data.get('buyer_name', ''),
            data.get('buyer_address', ''),
            data.get('buyer_contact', ''),
            data.get('receipt_date', ''),
            data.get('store_name', ''),
            data.get('store_address', ''),
            data.get('total_amount', ''),
            json.dumps(data.get('items', []), ensure_ascii=False)
        ]
        sheet.append_row(row_to_append, value_input_option='USER_ENTERED')
        return True
    except gspread.exceptions.APIError as e:
         st.error(f"Google Sheets API error during append: {e}", icon="❌")
         return False
    except Exception as e:
        st.error(f"Failed to append data to Google Sheet: {e}", icon="❌")
        return False

# --- Initialize OCR/Parser Clients (Paddle, LlamaParse remain the same) ---
@st.cache_resource
def get_paddle_ocr():
    try:
        paddle_ocr_instance = PaddleOCR(use_angle_cls=True, lang='vi', use_gpu=False, show_log=False)
        print("PaddleOCR initialized successfully for Vietnamese.")
        return paddle_ocr_instance
    except Exception as e:
        st.warning(f"Could not initialize PaddleOCR: {e}. PaddleOCR option will be disabled.", icon="⚠️")
        return None

@st.cache_resource
def get_llama_parser():
    if LLAMAPARSE_API_KEY:
        try:
            parser_instance = LlamaParse(api_key=LLAMAPARSE_API_KEY, result_type="text")
            print("LlamaParse parser initialized successfully.")
            return parser_instance
        except Exception as e:
             st.warning(f"Could not initialize LlamaParse: {e}. LlamaParse option disabled.", icon="⚠️")
             return None
    else:
        return None

paddle_ocr_instance = get_paddle_ocr()
llama_parser_instance = get_llama_parser()

# --- Streamlit App UI ---
st.title("🧾 Vietnamese Receipt OCR & Data Extraction (OpenRouter LLM)") # Updated Title
st.info(f"Uses Colab Auth for GSheets & OpenRouter ({DEFAULT_OPENROUTER_MODEL}) for LLM extraction.")

# Initialize session state (remains the same)
# ... (ocr_text, extracted_data, etc.) ...
if 'ocr_text' not in st.session_state: st.session_state.ocr_text = None
if 'extracted_data' not in st.session_state: st.session_state.extracted_data = None
if 'file_processed' not in st.session_state: st.session_state.file_processed = False
if 'confirmed_data' not in st.session_state: st.session_state.confirmed_data = None


# --- Sidebar for Configuration ---
with st.sidebar:
    st.header("Configuration")
    uploaded_file = st.file_uploader("Upload Receipt (Image or PDF)", type=["png", "jpg", "jpeg", "pdf"])

    # (OCR Method Selection - Remains the same)
    ocr_options = ['Tesseract (Local)']
    if paddle_ocr_instance: ocr_options.insert(0, 'PaddleOCR (Local)')
    else: st.warning("PaddleOCR option disabled (initialization failed).", icon="⚠️")
    if llama_parser_instance: ocr_options.append('LlamaParse (API)')
    elif LLAMAPARSE_API_KEY: st.warning("LlamaParse option disabled (initialization failed).", icon="⚠️")


    default_ocr_index = 0 # Default to 0 (Tesseract)
    tesseract_label = 'Tesseract (Local)'
    if tesseract_label in ocr_options:
        try:
            default_ocr_index = ocr_options.index(tesseract_label)
        except ValueError:
            pass # Should not happen as we always add it, but safer


    if not ocr_options:
         st.error("No OCR engines available!", icon="❌")
         ocr_method = None
    else:
         default_ocr_index = 0
         ocr_method = st.radio("Choose OCR Method:", options=ocr_options, index=default_ocr_index)

    run_ocr = st.button("1. Run OCR", disabled=(uploaded_file is None or ocr_method is None), use_container_width=True)

    st.markdown("---")
    # --- MODIFIED: LLM Extraction Button Check ---
    openrouter_ready = bool(openrouter_client) # Check if client initialized successfully
    extract_button_disabled = (not st.session_state.get('ocr_text') or not openrouter_ready)
    run_extraction = st.button("2. Extract Data with LLM",
                               disabled=extract_button_disabled,
                               use_container_width=True)
    if not openrouter_ready:
         st.error("OpenRouter API Key missing or invalid. LLM Extraction disabled.", icon="❗")
    # --- End Modification ---

    st.markdown("---")
    # (Google Sheet Config Info - Remains the same)
    st.header("Google Sheet Export")
    gsheet_ready_for_export = False # Default to false
    if GOOGLE_SHEET_NAME:
        st.info(f"Target Sheet: '{GOOGLE_SHEET_NAME}'", icon="📄")
        client_check = get_gspread_client() # Check connection status
        gsheet_ready_for_export = client_check is not None

    else:
        st.info("No Google Sheet name configured.", icon="ℹ️")


# --- Main Area ---
# (OCR Execution logic remains the same)
# (Extracted Data & Confirmation Form logic remains the same, but relies on the modified extract_data_with_llm)
col1, col2 = st.columns(2)

with col1:
    # Display Uploaded File & Run OCR Logic
    st.subheader("Uploaded File & OCR Text")
    if uploaded_file is not None:
        file_bytes = uploaded_file.getvalue()
        file_name = uploaded_file.name
        file_type = uploaded_file.type

        if file_type.startswith("image"):
            try:
                st.image(file_bytes, caption="Uploaded Receipt Image", use_column_width=True)
            except Exception as img_e:
                st.warning(f"Could not display image preview: {img_e}", icon="⚠️")
        elif file_type == "application/pdf":
            st.info(f"Uploaded PDF: {file_name}.", icon="📄")

        if run_ocr and ocr_method:
            # Reset states before running
            st.session_state.ocr_text = None
            st.session_state.extracted_data = None
            st.session_state.confirmed_data = None
            st.session_state.file_processed = True # Mark that processing was attempted
            ocr_output = None # Temp variable

            with st.spinner(f"Running {ocr_method}..."):
                if ocr_method == 'Tesseract (Local)':
                    if file_type.startswith("image"): ocr_output = process_image_tesseract(file_bytes)
                    elif file_type == "application/pdf": ocr_output = process_pdf_tesseract(file_bytes)
                elif ocr_method == 'PaddleOCR (Local)':
                    if file_type.startswith("image"): ocr_output = process_image_paddle(file_bytes, paddle_ocr_instance)
                    elif file_type == "application/pdf": ocr_output = process_pdf_paddle(file_bytes, paddle_ocr_instance)
                elif ocr_method == 'LlamaParse (API)':
                     ocr_output = process_file_llamaparse(file_bytes, file_name, llama_parser_instance)

            st.session_state.ocr_text = ocr_output # Assign result to session state

            if st.session_state.ocr_text and st.session_state.ocr_text.strip():
                st.success("OCR Completed!", icon="✅")
                # --- Force rerun to update button state ---
                st.rerun()
                # --- End force rerun ---
            else:
                st.error("OCR failed or produced no text.", icon="❌")
                # No rerun needed if OCR failed, button should remain disabled
        # --- End Run OCR Logic Modification ---

    if st.session_state.ocr_text:
        st.text_area("OCR Output", st.session_state.ocr_text, height=350, key="ocr_output_area")
    elif st.session_state.file_processed and not st.session_state.ocr_text:
          st.warning("No OCR text was generated from the file.", icon="⚠️")

with col2:
    st.subheader("Extracted Data & Confirmation")

    if run_extraction and st.session_state.ocr_text:
        st.session_state.extracted_data = None
        st.session_state.confirmed_data = None
        # --- MODIFIED: Spinner Message ---
        with st.spinner(f"Calling LLM via OpenRouter ({DEFAULT_OPENROUTER_MODEL})..."):
        # --- End Modification ---
            extracted_result = extract_data_with_llm(st.session_state.ocr_text) # This now calls the OpenRouter version
            st.session_state.extracted_data = extracted_result
        if isinstance(st.session_state.extracted_data, dict) and st.session_state.extracted_data:
            st.write("Review and edit the extracted data below:")

        if st.session_state.extracted_data:
             if "raw_llm_output" in st.session_state.extracted_data:
                 # Error message shown in extract_data_with_llm
                 st.warning("LLM did not return valid JSON. Cannot populate form.", icon="⚠️")
                 st.session_state.extracted_data = {} # Use empty dict
             else:
                 st.success("Data Extraction Attempted by LLM.", icon="🤖")
                 st.write("Review and edit the extracted data below:")
        else:
             # Error messages shown in extract_data_with_llm
             # st.error("LLM Data Extraction Failed.", icon="❌") # Redundant
             st.session_state.extracted_data = {} # Use empty dict

    # (Confirmation Form logic - No Changes needed here, just uses the result from extract_data_with_llm)
    if isinstance(st.session_state.extracted_data, dict) and st.session_state.extracted_data:

        with st.form("confirmation_form"):
            # ... (Form fields: text_input, text_area, date_input, number_input - remain the same) ...
            st.write("### Confirm Extracted Details")
            form_data = st.session_state.extracted_data.copy()
            c1, c2 = st.columns(2)
            with c1:
                form_data['buyer_name'] = st.text_input("Buyer Name", value=form_data.get('buyer_name', ''))
                form_data['buyer_address'] = st.text_area("Buyer Address", value=form_data.get('buyer_address', ''), height=100)
                form_data['buyer_contact'] = st.text_input("Buyer Contact", value=form_data.get('buyer_contact', ''))
                default_date_str = form_data.get('receipt_date', '')
                default_date = None
                if default_date_str:
                    try: default_date = datetime.datetime.strptime(default_date_str, '%Y-%m-%d').date()
                    except (ValueError, TypeError): st.warning(f"LLM date '{default_date_str}' not YYYY-MM-DD. Please verify.", icon="⚠️")
                form_data['receipt_date'] = st.date_input("Receipt Date", value=default_date)

            with c2:
                form_data['store_name'] = st.text_input("Store Name", value=form_data.get('store_name', ''))
                form_data['store_address'] = st.text_area("Store Address", value=form_data.get('store_address', ''), height=100)
                default_total_val = form_data.get('total_amount')
                default_total_float = 0.0
                if default_total_val is not None:
                    try:
                        if isinstance(default_total_val, str): cleaned_val = default_total_val.replace('.', '').replace(',', '')
                        else: cleaned_val = default_total_val
                        default_total_float = float(cleaned_val)
                    except (ValueError, TypeError): st.warning(f"Could not parse total amount: '{default_total_val}'. Defaulting to 0.0.", icon="⚠️")
                form_data['total_amount'] = st.number_input("Total Amount (VND)", value=default_total_float, format="%.0f", step=1.0)

            # ... (Items DataFrame and st.data_editor - remain the same) ...
            st.write("### Items Purchased")
            items_list = form_data.get('items', [])
            if not isinstance(items_list, list):
                 st.warning(f"Items data is not a list (found {type(items_list)}). Displaying empty editor.", icon="⚠️")
                 items_list = []
            try:
                processed_items = []
                required_item_keys = ['description', 'quantity', 'price']
                for item in items_list:
                    if isinstance(item, dict):
                         processed_item = {key: item.get(key) for key in required_item_keys}
                         processed_items.append(processed_item)
                    else: st.warning(f"Skipping invalid item data: {item}", icon="⚠️")
                items_df = pd.DataFrame(processed_items)
                for col in required_item_keys:
                     if col not in items_df.columns: items_df[col] = pd.NA
                items_df['description'] = items_df['description'].astype(str).fillna('')
                items_df['quantity'] = pd.to_numeric(items_df['quantity'], errors='coerce').fillna(0).astype(int)
                items_df['price'] = pd.to_numeric(items_df['price'], errors='coerce').fillna(0.0).astype(float)
                items_df = items_df[required_item_keys]
            except Exception as df_err:
                st.error(f"Could not create/process DataFrame from items list: {df_err}.", icon="❌")
                items_df = pd.DataFrame(columns=required_item_keys)
            edited_items_df = st.data_editor(
                items_df, num_rows="dynamic", column_config={
                     "quantity": st.column_config.NumberColumn("Qty", format="%d", step=1, min_value=0),
                     "price": st.column_config.NumberColumn("Price (VND)", format="%.0f", step=1.0, min_value=0.0),
                     "description": st.column_config.TextColumn("Description", width="large", required=True)},
                key="items_editor", use_container_width=True)
            form_data['items'] = edited_items_df.to_dict('records')

            # ... (Form Submission Button and Logic - remain the same, call connect_to_gsheet and append_to_gsheet) ...
            submit_button = st.form_submit_button(
                 "Confirm & Save to Google Sheet", disabled=not gsheet_ready_for_export, use_container_width=True)
            if submit_button:
                if form_data.get('receipt_date'): form_data['receipt_date'] = form_data['receipt_date'].strftime('%Y-%m-%d')
                else: form_data['receipt_date'] = ''
                form_data['total_amount'] = form_data.get('total_amount', 0.0)
                st.session_state.confirmed_data = form_data
                st.write("Data confirmed. Attempting to save to Google Sheet...")
                sheet, _ = connect_to_gsheet()
                if sheet:
                    with st.spinner("Appending data to Google Sheet..."):
                        success = append_to_gsheet(sheet, st.session_state.confirmed_data)
                        if success: st.success(f"Data successfully appended to Google Sheet '{GOOGLE_SHEET_NAME}'!", icon="✅")
                        else: st.error("Failed to save data to Google Sheet.", icon="❌")
                else: st.error("Cannot save data. Google Sheet connection failed.", icon="❌")

    elif run_extraction and not st.session_state.ocr_text:
         st.warning("Please run OCR first to generate text for extraction.", icon="⚠️")
    elif not st.session_state.extracted_data:
        st.info("Extracted data will appear here after running LLM extraction.")

Overwriting app.py


In [ ]:
# @title 4. Launch app via ngrok tunnel
from pyngrok import ngrok
import os
from dotenv import load_dotenv

load_dotenv()
NGROK_AUTH_TOKEN = os.getenv("NGROK_AUTH_TOKEN")

if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("Ngrok auth token set.")
else:
    print("Ngrok auth token not found in .env. Running without token.")

import subprocess
import threading

def run_streamlit():
    subprocess.run(['streamlit', 'run', '--server.port', '8501', '--server.headless=true', 'app.py'])

thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

public_url = None
try:
    public_url = ngrok.connect(addr='8501', proto='http')
    print(f"🎉 Your Streamlit app should be available at: {public_url}")
except Exception as e:
    print(f"❌ Error starting ngrok: {e}")
    print("   Streamlit might be running locally but ngrok tunnel failed.")

import time
try:
    while thread.is_alive():
        time.sleep(60)
    print("Streamlit thread seems to have stopped.")
except KeyboardInterrupt:
    print("\nKeyboardInterrupt received. Shutting down...")
except Exception as e:
    print(f"Exception in keep-alive loop: {e}")
finally:
    print("Closing ngrok tunnel...")
    if public_url:
        try: ngrok.disconnect(public_url); print("Ngrok tunnel disconnected.")
        except Exception as ng_e: print(f"Error disconnecting ngrok: {ng_e}")
    ngrok.kill()
    print("Ngrok process killed.")
    print("Exiting keep-alive loop.")

Ngrok auth token set.
🎉 Your Streamlit app should be available at: NgrokTunnel: "https://654a-34-83-108-135.ngrok-free.app" -> "http://localhost:8501"

KeyboardInterrupt received. Shutting down...
Closing ngrok tunnel...
Error disconnecting ngrok: ngrok client exception, URLError: [Errno 111] Connection refused
Ngrok process killed.
Exiting keep-alive loop.
